In [123]:
# IMPORTS
import pandas as pd
from os.path import join
import matplotlib.pyplot as plt
from statsmodels.tsa.statespace.sarimax import SARIMAX
import numpy as np
from concurrent.futures import ProcessPoolExecutor
from multiprocessing import Pool
from prophet import Prophet


In [124]:
#Load the data

def loadData(path, country, date_from, date_to, date_format="%Y-%m-%d %H:%M:%S"):
        # LOAD TABLES
    consumptions_path = join(path, "historical_metering_data_" + country + ".csv")
    features_path = join(path, "spv_ec00_forecasts_es_it.xlsx")
    consumptions = pd.read_csv(consumptions_path, index_col=0, parse_dates=True, date_format=date_format)
    features = pd.read_excel(features_path, sheet_name=country, index_col=0, parse_dates=True, date_format=date_format,)
        # MODIFY COLUMNS
    features = features.reset_index()
    features = features.rename(columns={"index": "time"})
    features = features[(features['time'] >= date_from) & (features['time'] <= date_to)]
    consumptions['total_consumption'] = consumptions.sum(axis=1)
    consumptions = consumptions.reset_index()
    consumptions = consumptions.rename(columns={"DATETIME": "time"})
    consumptions = consumptions[(consumptions['time'] >= date_from) & (consumptions['time'] <= date_to)]
        # LOAD ROLLOUTS
    features2 = pd.read_csv(path+"/rollout_data_"+country+".csv")
    features2 = features2.rename(columns={"DATETIME": "time"})
    features2['time'] = pd.to_datetime(features2['time'])
    features2 = features2[(features2['time'] >= date_from) & (features2['time'] <= date_to)]
        # LOAD HOLIDAYS
    holidays = pd.read_excel(path+"/holiday_"+country+".xlsx")["holiday_"+country]
        # CREATE TIME FEATURES
    features3 = features.copy()
    features3 = features3[["time"]]
    holidays = pd.to_datetime(holidays)
    features3['day_nr_inc'] = (features3['time'] - date_from).dt.days
    features3['is_holiday'] = features3['time'].dt.date.isin(holidays.dt.date)
    features3['is_weekend'] = features3['time'].dt.dayofweek >= 5
    features3['month'] = features3['time'].dt.month
    features3['week'] = features3['time'].dt.isocalendar().week
    features3['day_of_week'] = features3['time'].dt.dayofweek + 1
    features3['year'] = features3['time'].dt.year
    # Create binary variables for each day of the week (1-7)
    for day in range(1, 8):
        features3[f'day_{day}'] = (features3['time'].dt.dayofweek + 1 == day).astype(int)
    # Create binary variables for each month of the year (1-12)
    for month in range(1, 13):
        features3[f'month_{month}'] = (features3['time'].dt.month == month).astype(int)
    # CREATE FINAL FEATURE DATASET
    features = features.merge(features2, on="time", how="left").merge(features3, on="time", how="left")
    # DETERMINE CUSTOMERS
    customer_names = []
    for column_name in consumptions.columns:
        if "VALUEMWH" in column_name:
            customer_names.append("_".join(column_name.split("_")[1:]))
    # Return
    return consumptions, features, customer_names

In [125]:
def getDataForCustomer(customer, consumptions, features):
    Y = consumptions[["time", "VALUEMWHMETERINGDATA_"+customer]]
    Y = Y.rename(columns={"VALUEMWHMETERINGDATA_"+customer:"consumption"})
    X = features[["time", "spv", "temp", "INITIALROLLOUTVALUE_"+customer, "day_nr_inc", "is_holiday", "is_weekend", "month", "week", "day_of_week", "year", "day_1", "day_2", "day_3", "day_4", "day_5", "day_6", "day_7", "month_1", "month_2", "month_3", "month_4", "month_5", "month_6", "month_7", "month_8", "month_9", "month_10", "month_11", "month_12",]]
    return Y, X

In [126]:
def analyseDataConsistency(Y):
        # Find the first non-NaN value in 'consumption'
    first_non_nan_index = Y['consumption'].first_valid_index()
    first_non_nan_date = Y.loc[first_non_nan_index, 'time']
        # Check if there are any NaN values later in the column
    has_nan_later = Y['consumption'].isnull().any()
    return first_non_nan_date, has_nan_later

In [127]:
# PARAMETERS
input_data_path = "../../data/1_original/OneDrive_2025-04-05/Alpiq ETHdatathon challenge 2025/datasets2025/"
output_data_path = "../../data/2_processed/"
countries = ["IT", "ES"]
training_date_from = pd.Timestamp("2023-07-01 00:00:00")
# training_date_to = pd.Timestamp("2022-05-31 23:00:00")
training_date_to = pd.Timestamp("2024-07-31 23:00:00")
forecast_steps = 24*31
max_threads = 4  # Maximum number of threads to run in parallel
country = "IT"

In [128]:
# STEP 1: LOAD DATA
consumptions, features, customer_names = loadData(input_data_path, country, training_date_from, training_date_to, date_format="%Y-%m-%d %H:%M:%S")
customer = customer_names[0]



# STEP 2: CLEAN DATA
Y, X = getDataForCustomer(customer, consumptions, features)

In [129]:
# Print the first few rows of Y and X
print("Y:")
print(Y.head())
print("\nX:")
print(X.head())


Y:
                     time  consumption
13102 2023-07-01 00:00:00     0.001689
13103 2023-07-01 01:00:00     0.001605
13104 2023-07-01 02:00:00     0.001558
13105 2023-07-01 03:00:00     0.001647
13106 2023-07-01 04:00:00     0.001595

X:
                 time  spv  temp  INITIALROLLOUTVALUE_customerIT_1  \
0 2023-07-01 00:00:00  0.0  18.7                             0.007   
1 2023-07-01 01:00:00  0.0  18.2                             0.007   
2 2023-07-01 02:00:00  0.0  17.8                             0.006   
3 2023-07-01 03:00:00  0.0  17.5                             0.006   
4 2023-07-01 04:00:00  0.0  17.4                             0.006   

   day_nr_inc  is_holiday  is_weekend  month  week  day_of_week  ...  month_3  \
0           0       False        True      7    26            6  ...        0   
1           0       False        True      7    26            6  ...        0   
2           0       False        True      7    26            6  ...        0   
3           0 

In [130]:
for col in X.columns:
    print(col)

time
spv
temp
INITIALROLLOUTVALUE_customerIT_1
day_nr_inc
is_holiday
is_weekend
month
week
day_of_week
year
day_1
day_2
day_3
day_4
day_5
day_6
day_7
month_1
month_2
month_3
month_4
month_5
month_6
month_7
month_8
month_9
month_10
month_11
month_12


In [131]:
def train_prophet_model(Y, X):
    # Focus primarily on recent data
    one_year_ago = Y['time'].max() - pd.Timedelta(days=365)
    
    # Use 80% recent data, 20% older data
    Y_recent = Y[Y['time'] >= one_year_ago].copy()
    X_recent = X[X['time'] >= one_year_ago].copy()
    
    # Sample some older data (20% of the recent data size)
    Y_older = Y[Y['time'] < one_year_ago].sample(frac=0.2)
    X_older = X[X['time'].isin(Y_older['time'])].copy()
    
    # Combine with emphasis on recent data
    Y_weighted = pd.concat([Y_recent, Y_older])
    X_weighted = pd.concat([X_recent, X_older])

    # *** USE THE WEIGHTED DATA HERE ***
    # Prepare the data for Prophet
    df = Y_weighted.rename(columns={"time": "ds", "consumption": "y"})
    df['y'] = df['y'].fillna(0)  # Fill NaN values with 0
    
    # Merge the regressors from X_weighted into df
    X_temp = X_weighted.copy()
    if 'time' in X_temp.columns:
        X_temp = X_temp.rename(columns={"time": "ds"})
    
    # Merge X_weighted into df on the 'ds' column
    df = df.merge(X_temp, on='ds', how='left')
    
    # Create a Prophet model
    model = Prophet()
    
    # Add regressors from X to the model
    # for col in ['spv', 'temp', 'INITIALROLLOUTVALUE_customerIT_1', 'day_nr_inc', 'is_holiday', 'is_weekend']:
    #     model.add_regressor(col)
    
    # Fit the model
    model.fit(df)
    
    return model

# Train the model
prophet_model = train_prophet_model(Y, X)

17:58:39 - cmdstanpy - INFO - Chain [1] start processing
17:58:39 - cmdstanpy - INFO - Chain [1] done processing


In [132]:
print(X.columns)

Index(['time', 'spv', 'temp', 'INITIALROLLOUTVALUE_customerIT_1', 'day_nr_inc',
       'is_holiday', 'is_weekend', 'month', 'week', 'day_of_week', 'year',
       'day_1', 'day_2', 'day_3', 'day_4', 'day_5', 'day_6', 'day_7',
       'month_1', 'month_2', 'month_3', 'month_4', 'month_5', 'month_6',
       'month_7', 'month_8', 'month_9', 'month_10', 'month_11', 'month_12'],
      dtype='object')


In [136]:
# Make future predictions
future = prophet_model.make_future_dataframe(periods=forecast_steps, freq='H')

# Extract the unique dates from the future dataframe
future_dates = future['ds'].unique()

# Prepare regressor data for future dates
future_regressors = X.copy()
future_regressors = future_regressors.rename(columns={"time": "ds"})
future_regressors = future_regressors[future_regressors['ds'].isin(future_dates)]

# Merge the regressors into the future DataFrame
future = future.merge(future_regressors, on='ds', how='left')

# Fill NaN values in the regressor columns
for column in X.columns:
    if column != 'time':
        future[column] = future[column].fillna(0)  # Or use mean: X[column].mean()

# Now predict with the complete future DataFrame
forecast = prophet_model.predict(future)

# make all negative values zero
forecast['yhat'] = forecast['yhat'].clip(lower=0)

/home/epsilon/anaconda3/envs/spyder-env/lib/python3.13/site-packages/prophet/forecaster.py:1854: FutureWarning:

'H' is deprecated and will be removed in a future version, please use 'h' instead.



In [137]:
print(forecast.head())
print(forecast.tail())

                   ds     trend  yhat_lower  yhat_upper  trend_lower  \
0 2023-07-01 00:00:00  0.004788   -0.035356    0.006369     0.004788   
1 2023-07-01 01:00:00  0.004790   -0.031224    0.008118     0.004790   
2 2023-07-01 07:00:00  0.004797   -0.022948    0.018125     0.004797   
3 2023-07-01 13:00:00  0.004805   -0.014976    0.026415     0.004805   
4 2023-07-01 15:00:00  0.004807   -0.005722    0.032938     0.004807   

   trend_upper  additive_terms  additive_terms_lower  additive_terms_upper  \
0     0.004788       -0.017739             -0.017739             -0.017739   
1     0.004790       -0.016041             -0.016041             -0.016041   
2     0.004797       -0.006881             -0.006881             -0.006881   
3     0.004805        0.000952              0.000952              0.000952   
4     0.004807        0.009161              0.009161              0.009161   

      daily  daily_lower  daily_upper    weekly  weekly_lower  weekly_upper  \
0 -0.014741    -0.0

In [141]:
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

# Rename forecast columns to match your naming convention
forecast_renamed = forecast.rename(columns={"ds": "time", "yhat": "Y"})

# Create figure with subplots
fig = make_subplots(rows=2, cols=1, subplot_titles=("DATA AND PREDICTION", "RESIDUAL ERROR OF MODEL"))

# Add traces to first subplot
fig.add_trace(
    go.Scatter(x=Y["time"], y=Y['consumption'], name="Historical Data"),
    row=1, col=1
)
fig.add_trace(
    go.Scatter(x=forecast_renamed["time"], y=forecast_renamed["Y"], name="Forecast", line=dict(color='red')),
    row=1, col=1
)


# Calculate and plot residuals for the overlapping period
merged_data = pd.merge(Y, forecast_renamed, on='time', how='inner')
residuals = merged_data['consumption'] - merged_data['Y']
fig.add_trace(
    go.Scatter(x=merged_data["time"], y=residuals, name="Residuals"),
    row=2, col=1
)
print(np.sum(np.abs(residuals)))

# Update layout
fig.update_layout(height=800, width=1000, title_text=f"{country}_{customer} Forecast")
fig.update_xaxes(title_text="Time", row=2, col=1)
fig.update_yaxes(title_text="Consumption", row=1, col=1)
fig.update_yaxes(title_text="Error", row=2, col=1)

# Save as interactive HTML
fig.write_html(f"{country}_{customer}_forecast_interactive.html")

# Display in notebook
fig.show()

93.22138409726936
